In [6]:
"""
This script applies the exponential local recession curve watertable fluctuation method (e.g., Solorzano-Rivas
et al. 2024) to estimate recharge from synthetic hydrographs. The method is implemented on a series of generated 
recharge events, and recharge estimates are compared to known values. The output is the relative error (%) of the 
recharge estimate, calculated as described in Becke et al. (2025).
"""
import os
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from pathlib import Path

# Configuration
BASE_DIR = Path.home() / "workspace" / "WTF" / "OutputFiles" 
INPUT_DIR = BASE_DIR / "data"
OUTPUT_DIR = BASE_DIR / "ExpLRC"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# --- CONSTANTS ---
time = np.linspace(0, 400., 401)
t1, t2, t3 = 30, 90, 120
t4, t5, t6, t7 = 210, 240, 360, 390

# --- FUNCTION ---
def compute_lrc_errors(itr, time, t3, t4, t5, input_path):
    errors = []

    time_to_fit = time[t3:t4 + 1]
    time_to_extend = time[t3:t5 + 1]

    for j in range(1, itr + 1):
        try:
            wl_file = input_path / f"hydro_{j:04d}.csv"
            var_file = input_path / f"var_{j:04d}.csv"

            df_hydro = pd.read_csv(wl_file, header=None).to_numpy().flatten()
            df_var = pd.read_csv(var_file, header=None).to_numpy()

            wl_segment = df_hydro[t3:t4 + 1]

            def single_exp_model(t, A, B):
                h0 = wl_segment[0]
                return (B / A) - np.exp(-A * (t - time_to_fit[0])) * ((-A * h0) + B) / A

            params, _ = curve_fit(
                single_exp_model, time_to_fit, wl_segment,
                maxfev=50000, method='trf'
            )
            C1, C2 = params

            projected = single_exp_model(time_to_extend, C1, C2)
            delta_h = df_hydro[t5] - projected[-1]
            delta_t = time[t5] - time[t4]

            recharge_est = (delta_h / delta_t) * df_var[3][0]
            true_recharge = df_var[0][0]
            error_pct = ((recharge_est - true_recharge) / true_recharge) * 100
            errors.append(error_pct)

        except Exception as e:
            print(f"[Warning] Skipping hydrograph {j:04d}: {e}")
            continue

    return errors

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    event_configs = {
        1: (t1, t2, t3),
        2: (t3, t4, t5),
        3: (t5, t6, t7)
    }

    for event_id, (fit_start, fit_end, project_end) in event_configs.items():
        errors_empirical = compute_lrc_errors(
            itr=1000,
            time=time,
            t3=fit_start,
            t4=fit_end,
            t5=project_end,
            input_path=INPUT_DIR
        )

        output_file = OUTPUT_DIR / f"RE{event_id}_LRC_Exp_RError.csv"
        np.savetxt(
            output_file,
            errors_empirical,
            delimiter=',',
            header=f"LRC_Exp_Rerror_Event{event_id}",
            comments=''
        )
        print(f"Saved recharge errors for event {event_id} to: {output_file}")


Saved recharge errors for event 1 to: C:\Users\beck0213\workspace\WTF\OutputFiles\GITHUB_CHECK\ExpLRC\RE1_LRC_Exp_RError.csv
Saved recharge errors for event 2 to: C:\Users\beck0213\workspace\WTF\OutputFiles\GITHUB_CHECK\ExpLRC\RE2_LRC_Exp_RError.csv
Saved recharge errors for event 3 to: C:\Users\beck0213\workspace\WTF\OutputFiles\GITHUB_CHECK\ExpLRC\RE3_LRC_Exp_RError.csv
